## Ajax
* Asynchrounous Javascript And Xml
* With using ajax, html doesn't contain whole tags and more things
* JS gonna render
* JS gonna append and insert the tags 
* These make user can use the web before whole rending >> fast

### Congressman data web scraping
<pre>    
    Scrap the data
->  Store to pandas dataframe
->  Plotting(bar plot, histogram, pychart)
->  Store to DB
</pre>


In [2]:
# https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do


### Developer tool >> Network tap >> .do file >> form data !! ############

# currentPage: 13
# currentPage: 7
# rowPerPage: 6
#  ...
##########################################################################

### >> query! :: ?currentPage=1&rowPerPage=300 (300 rows!)
# https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do?currentPage=1&rowPerPage=300

In [3]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests,re
from urllib.parse import urljoin

def cleaning_str(some_str):
    regex = re.compile(r'[\n\r\t]')
    result = regex.sub('',some_str)
    return result.strip().replace(' ' ,'')

url = 'https://www.assembly.go.kr/assm/memact/congressman/memCond/memCondListAjax.do?currentPage=1&rowPerPage=300'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

men_list = soup.select('.memberna_list dl dt a')
# print(len(men_list))
men_dicts = []
for men in men_list:
    name = men.string
    matched = re.search(r"(\d+)", men['href'])
    link = f"https://www.assembly.go.kr/assm/memPop/memPopup.do?dept_cd={matched.group(0)}"
    # print(f'{name} {link}')
    men_dicts.append({
        'name' : name,
        'link' : link,
    })
congressman = []
for link in men_dicts:
    html = requests.get(link['link']).text
    soup = BeautifulSoup(html, 'html.parser')
    # print(link['name'], end = ' ')
    birth_day = ''
    img_url =''
    party = ''
    elec = ''
    dt = []
    dd = []
    for i in soup.select('.pro_detail dd'): ## data
        if i.string is None:
            dd.append("NULL")
        else:
            dd.append(cleaning_str(i.string)) #
    for i in soup.select('.pro_detail dt'): ## title 
        dt.append(i.string)
    congressman_dict = dict(zip(dt,dd))
    for i in soup.select('.profile'):
        congressman_dict['이름'] = link['name']
        congressman_dict['생년월일'] = i.select_one('li:nth-of-type(4)').string
        congressman_dict['image'] = urljoin(link['link'], i.select_one("img")['src'])

    congressman.append(congressman_dict)
    print(f'{link["name"]}\'s information has appended')
print('Finished!')
    # print(congressman_dict)
# data = pd.DataFrame.from_dict(congressman)
# data

강기윤's information has appended
강대식's information has appended
강득구's information has appended
강민국's information has appended
강민정's information has appended
강병원's information has appended
강선우's information has appended
강은미's information has appended
강준현's information has appended
강훈식's information has appended
고민정's information has appended
고영인's information has appended
고용진's information has appended
곽상도's information has appended
구자근's information has appended
권명호's information has appended
권성동's information has appended
권영세's information has appended
권은희's information has appended
권인숙's information has appended
권칠승's information has appended
기동민's information has appended
김경만's information has appended
김경협's information has appended
김교흥's information has appended
김기현's information has appended
김남국's information has appended
김도읍's information has appended
김두관's information has appended
김미애's information has appended
김민기's information has appended
김민석's information has appended
김민철's in

In [4]:
data = pd.DataFrame.from_dict(congressman)
data.index = np.arange(1, len(data)+1)
# data.ignore_index()
# spe = data['취미, 특기'] != 'NULL'
# data.loc[spe,:]
# data.shape
# data['정당'].unique()
data['정당'].value_counts()
data['선거구'].value_counts()
data['당선횟수'].value_counts()
# data['소속위원회'].value_counts()
# data.info()
data['소속위원회'].value_counts()
# data.info()
data['생년월일']

1      1960-06-04
2      1959-11-02
3      1963-05-27
4      1971-03-03
5      1961-04-26
          ...    
296    1978-11-24
297    1954-12-05
298    1976-08-05
299    1962-09-10
300    1967-07-28
Name: 생년월일, Length: 300, dtype: object

In [5]:
data['당선횟수'] = data['당선횟수'].str[:2] ### Series object to string >> this makes can sliciing
# data['당선횟수'] = elec[:2]
# data['당선횟수']
print(data['당선횟수'].value_counts())
data.loc[data['당선횟수'] == '6선',['이름','정당']]

초선    151
재선     74
3선     42
4선     19
5선     13
6선      1
Name: 당선횟수, dtype: int64


,이름,정당
93,박병석,무소속


In [26]:
# year = data['생년월일']
age_max = data['생년월일'].str[:4].max()
age_min = data['생년월일'].str[:4].min()
#  data['생년월일']
print(data.loc[data['생년월일'].str[:4] == age_max,'이름'])
print("=" * 100)
print(data.loc[data['생년월일'].str[:4] == age_min,'이름'])
data['year'] = data['생년월일'].str[:4].astype(int)
data['month'] = data['생년월일'].str[5:7].astype(int)
data['day'] = data['생년월일'].str[8:10].astype(int)
data.loc[:,'year':'day']


81    류호정
Name: 이름, dtype: object
66     김진표
291    홍문표
Name: 이름, dtype: object


,year,month,day
1,1960,6,4
2,1959,11,2
3,1963,5,27
4,1971,3,3
5,1961,4,26
...,...,...,...
296,1978,11,24
297,1954,12,5
298,1976,8,5
299,1962,9,10


In [27]:
## User DatetimeIndex class [year/month/day] extracting
# birth_day = data['생년월일']
# str(birth_day.encode= 'utf-8')
# print(type(birth_day))
# data['year'] = pd.DatetimeIndex(birth_day).year

In [28]:
from datetime import date

def calc_age(dtob):
    today = date.today()
    return today.year - dtob.year - ((today.month, today.day) < (dtob.month, dtob.day))
for idx,row in data.iterrows():
    age = calc_age(date(row['year'], row['month'], row['day']))
    data.loc[idx,'age'] = int("{:2d}".format(age))


# print(data['age'].max())
# age(date(data['생년월일'].str[:4],data['생년월일'].str[5:7],data['생년월일'].str[8:10]))
# age(date(year=1993, month=10, day =10))

In [29]:
print('최고령 : ', data['age'].max())
print('최저령 : ', data['age'].min())
print('평균 나이 : ', int(data['age'].mean()))

temp = data['선거구'].str
data['선거구2'] = temp[:2]
data.loc[data['선거구2'] == '부산', '이름']
data['선거구2'].value_counts()
# data['선거구2'].value_counts(normalize=True) ## normalization

최고령 :  73
최저령 :  27
평균 나이 :  55


경기    59
서울    49
비례    47
부산    18
경남    16
인천    13
경북    13
대구    12
충남    11
전남    10
전북    10
광주     8
충북     8
강원     8
대전     7
울산     6
제주     3
세종     2
Name: 선거구2, dtype: int64